![HPEDEVlogo](Pictures/hpedevlogo-NB.JPG)    ![Dockerlogo](Pictures/docker.png)  

Let's start by connecting to our docker appliance again (Docker in Docker Appliance)

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}

In [ ]:
%login 16.31.86.200

# Configuring nextcloud in a container

Estimated time: 60 minutes.

Based on the work done in the Docker Dojo during a Grenoble Docker Meetup (cf: https://github.com/Enalean/docker-dojo/tree/master/owncloud).

Nextcloud is a web based application providing services such as calendar data or file sharing e.g.
When we want to contain an application such as nextcloud, there are a certain number of aspects to take in account and solve:
  1. installing the application and its dependencies in the container
  2. allow IP configuration for remote access to the application
  3. allow data persistence at each invocation of the container
  4. allow configuration data persistence at each invocation of the container
  
One possibility would be to run the container from an image and launch the various commands in the container (as we've done previously). We could put that in a script and launch it systematically when we instantiate a container from an image, or rebuild a prepared image to be instantiated later. But there is a better way to achieve what we want to do, and this is by using the automation process by Docker with the Dockerfile.

The Dockerfile is a way to describe all the operations required to create an image from an initial basic one and stacking all the operations to build at the end the final image ready to be instantiated and consumed and thrown away.

Let's start our Dockerfile by creating a simple container from a base image and just installing some software components useful for our environment, and build an image from that:


In [ ]:
echo 'FROM centos:7' > Dockerfile
echo 'RUN yum install -y httpd' >> Dockerfile
cat Dockerfile

In [ ]:
docker build .

In [ ]:
docker image ls

So we can verify that a new CentOS 7 image has been downloaded and based on it a new image has been created (without name nor tag, just an ID) containing httpd installed with its dependencies. 
Check it by instantiating a container based on that image and launching httpd in it:

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
docker run -d $ImgId /usr/sbin/httpd

In [ ]:
ps auxww |  grep http

Strange, no httpd daemon seems to be running whereas you had no error message reported. Let's try to understand why. Look at the containers running:

In [ ]:
docker container ls

Ok, so you don't have any container running. Look at one which exited then:

In [ ]:
docker container ls -a

So here it is ! But as seen previously the container exited with a 0 status code, meaning no error. Why ? Well this is because the httpd process is giving back hand to the user immediately after launch, going into background. So Docker thinks that all the tasks that he has to do where done correctly and exit gracefully. Assess that analysis by issuing the following commands:

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls -a | grep $ImgId | awk '{print $1}'`
docker diff $CtnId
echo ""
docker history $ImgId
echo ""
docker logs $CtnId

So we checked that we can launch the httpd server from inside an instantiated container made from our image. It created the relevant log files in the container, with just a normal warning. We also checked how our image was built. Note that the image built is around 170 MB larger than the base CentOS 7 one (shown by history) and has sensible modifications shown by the diff command.

It's a good start, but now we would like to have our httpd server started automatically with our container creation and staying alive after launch. And have attribution accordingly ;-) So let's make the following modifications to the Dockerfile

In [ ]:
echo 'MAINTAINER myself@mydomain.org' >> Dockerfile
echo 'CMD /usr/sbin/apachectl -DFOREGROUND -k start' >> Dockerfile
echo ""
cat Dockerfile

In [ ]:
docker build .

You can remark that all the first steps are very quick. This is because Docker caches steps, and will not repeat them unless the Dockerfile changes. You can modify the Dockerfile by putting the `MAINTAINER` command as the second line and re-launch the build. You'll see that in that case Docker invalidates its cache and restarts.
Secondly, we modified the way our Apache Web server is started by forcing it to stay in foreground after launch. That way Docker will continue to keep alive the container.
Now start a container from that image to check the web server is indeed started

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
docker run -d $ImgId

Now check using the same previous commands that the container is indeed running correctly:

In [ ]:
docker container ls
echo ""
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker diff $CtnId
echo ""
docker history $ImgId
echo ""
docker logs $CtnId
echo ""
ps auxww | grep httpd | grep -v docker

So you can see that now your httpd process is running, has produced more differences (a pid file has been created), more log content. It is also running with a UID (48) which is not known by the host, just the container. So now let's try to reach our newly launched webserver:

In [ ]:
curl http://localhost

Of course that doesn't work ;-) You would have been surprised otherwise don't you ? The major aspect here is that the httpd server you launched is run in an isolated environement, the container, that is NOT the host, so its localhost is different from the localhost of the host. Similarly its IP address is different from the one of the host. Proof:

In [ ]:
echo ""
echo "*** host ***"
echo ""
echo "IP: {{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}"
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
docker exec $CtnId yum install -y net-tools >& /dev/null
echo ""
echo -n "IP: "
docker exec $CtnId /usr/sbin/ifconfig | grep -A 2 eth0 | tail -2 | head -1 | awk '{print $2 }'

Convinced ? So now try *inside* the container to reach the Web server:

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
docker exec $CtnId curl -s http://localhost

Success !! So you indeed have a running web server, inside a CentOS 7 distribution container. But that's not fully what you want. What you want is to be able to reach it from outside of the container.

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
CtnIP=`docker exec $CtnId ifconfig | grep -A 2 eth0 | grep inet | awk '{print $2}'`
echo ""
echo "*** using CTN IP $CtnIP ***"
echo ""
curl -s http://$CtnIP | grep CentOS
HostIP="{{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}"
echo ""
echo "*** using host IP $HostIP ***"
echo ""
curl -s http://$HostIP | grep CentOS


So it works when you use the container IP (because Linux and Docker set the routing up for you automagically, but it doesn't work if you use the IP address of the host, which is really what you want to provide access to for the external world. )

By default, the container ports are not exposed outside of the container. So you can't use your host OS to access your isolated webserver. 

You will have to explicitly redirect trafic arriving on a specific port allocated to you on the host (which is accessible from outside your environment) to the container port 80 to allow access to the web server running in the container. This will require a change to the way you invoke the container creation. Let's stop the previous container and relaunch it with the right options:

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker stop $CtnId
docker rm $CtnId
docker run -d -p $webport:80 $ImgId
echo ""
docker container ls
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** using host IP ***"
echo ""
HostIP={{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}
curl http://$HostIP:$webport | grep CentOS

Now that we have redirected the port correctly, we can reach our webserver from outside the container using the host IP address. If that machine is reachable from the Internet, you have a Webserver in a container avaible online.

And as this is the case in our environment, please launch a Web browser of your choice and reach the URL mentioned in the next cell to check it.

In [ ]:
firefox http://{{ JPHOSTEXT }}:$webport

Don't you feel powerful now ?


It's now time to add some useful content to our web server !
Modify again the Dockerfile to add nextcloud to our image:

In [ ]:
echo 'RUN nextcloud' >> Dockerfile
echo 'RUN curl http://{{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}/nextcloud-22.1.1.zip -o /tmp/nextcloud.zip' >> Dockerfile
echo 'RUN cd /var/www/html/ && unzip -q /tmp/nextcloud.zip' >> Dockerfile

Now re-create a new image based on that Dockerfile, purge the previous content before relaunching a new container (please wait till the end of the operation as the download is around 160MB and can take some time)

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker stop $CtnId
docker rm $CtnId
echo ""
docker build .
echo ""
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
echo ""
docker run -d -p $webport:80 $ImgId
docker container ls

Try now to connect to your nextcloud instance. 

![Owncloud failed](Pictures/owncloud_without_dep.png)

1. What happens?
2. What should you do next to solve the issue ? **Discuss with your trainer if you're stuck !**

Hint, you probably need to add the nextcloud dependencies to be able to launch it. Add the following line to your Dockerfile to solve that:

In [ ]:
echo "RUN yum install -y php php-dom php-mbstring php-pdo php-gd" >> Dockerfile

With that you should be able to use owncloud ! (Note that you need to use that version with CentOS 7 for a PHP dependency management) But we're not done yet !!!
If you log on to your owncloud instance, and start customizing it (login/passwd for admin, storage path), you'll have errors first, that we'll fix later on and then if you `Docker stop` and `Docker rm` the container to relaunch it again, of course, none of this customization will be kept as it's not part of your container content.

So we now have to deal with storage management for our Docker container. First we need to solve the error generated when you tried to configure your nextcloud instance. We had rights issues. Use the following command to help solve the issue:

In [ ]:
docker exec b42f9f6f1034 ls -al /var/www/html
docker exec b42f9f6f1034 ps auxww | grep httpd

The principle is that the owner of the httpd process should have the rights on the nextcloud directory to read and store files there. ** So modify your Dockerfile accordingly and retest **.

Now you should be able to customize your owncloud instance and start using it.

By now you have probably remarked that you have to each time deal with IDs for containers and images, which is not that convenient. Let's fix that. Download the owncloud tar file in your directory and modify the ADD line:

In [ ]:
docker build -t nextcloud .

You now have tagged your image and use it by its name:

In [ ]:
docker image ls

It would be great if you could persist the content from one run to another.  Yes, you can ;-) For that, you need to attach a local directory of your host to your container, and point the setup of your owncloud to that directory instead of the one under `/var/www/html/owncloud`.
Create a `/data` directory on your host, mount it in your container under `/data`, and then point your setup ot it:

In [ ]:
mkdir -p /data

In [ ]:
date > /data/myfile.txt

In [ ]:
cat >> Dockerfile << EOF

In [ ]:
```
VOLUME /data
EOF
```

In [ ]:
docker build -t owncloud .

In [ ]:
docker ps

In [ ]:
docker stop 29c8f5ca3d76 

In [ ]:
docker rm 29c8f5ca3d76

In [ ]:
docker run -d -p 80:80 -v /data:/data owncloud:latest

Now reload the owncloud configuration page in your browser, but this time configure the data folder as in the following screen shot:

![Owncloud Setup](Pictures/owncloud.png)

** If you encounter issues you need to adapt your environment so that the apache user is allowed to write on to the /data directory. **

Your current Dockerfile should look like this at that point:

In [ ]:
cat Dockerfile

In [ ]:
```
FROM centos:6
#FROM fedora:latest
RUN yum install -y httpd
MAINTAINER myself@mydomain.org
RUN yum install -y tar bzip2
COPY owncloud-7.0.15.tar.bz2 /var/www/html/
RUN cd /var/www/html/ && tar xvfj owncloud-7.0.15.tar.bz2 && rm -f owncloud-7.0.15.tar.bz2
RUN yum install -y php php-dom php-mbstring php-pdo php-gd
VOLUME /data
RUN chown -R apache:apache /var/www/html/owncloud /data
CMD /usr/sbin/apachectl -DFOREGROUND -k start
EXPOSE 80
```

Move the example text file you created earlier to your ownClould Documents folder so you can see the file and view the file in ownCloud.

In [ ]:
mv /data/myfile.txt /data/bruno/files/Documents

Open the Documents folder in the ownCloud Web UI. Confirm that the myfile.txt example file is present and then view the contents to check that they match what you created earlier.


In [ ]:
docker ps

In [ ]:
docker stop 23f
docker rm 23f

In [ ]:
docker run -d -p XXXX:80 -v /data:/data owncloud:latest

1. At that point you should find again your data on your owncloud instance right ? But what additional pain point do you have ?
2. Knowing that the owncloud configuration data are located under `/var/www/html/owncloud/config/config.php`  try to adapt the Dockerfile to solve that last issue. **Discuss with your trainer if you're stuck !**
Note : there is more than one way to solve this.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#551199;"></i>&nbsp;&nbsp;Next Steps

# Lab4 : Using Docker Compose

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="4-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next LAB: Using Docker Compose"><i class="fas fa-chevron-circle-right" style="color:#551199;"></i></a></h2>

</br>
 <a href="1-WKSHP-Intro-to-Containers-techno.ipynb" target="New" title="Back: Introduction to Containers technologies"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="4-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next:Using Docker Compose"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
